In [3]:
from scene import Scene
import taichi as ti
from taichi.math import *
import numpy as np

%load_ext autoreload
%autoreload 2

In [5]:
@ti.func
def add_pokeball(r, pad, origin, color):
    for i, j, k in ti.ndrange((-r-pad, r+pad), (-r-pad, r+pad), (-r-pad, r+pad)):
        xyz = ivec3(i, j, k)
        if xyz.dot(xyz) < r**2: 
            scene.set_voxel(vec3(i, j, k) + origin, 1, color)

@ti.func
def origin_y(largest, r):
    return -(largest-r)-1

@ti.kernel
def initialize_voxels():
    add_pokeball(24, 1, vec3(-38, origin_y(largest, 24), 0),RED)
    add_pokeball(6, 1, vec3(17, origin_y(largest, 6), -2), WHITE)
    # add_pokeball(largest, 1, vec3(14, origin_y(largest, largest), -34),PURPLE)
    add_pokeball(12, 1, vec3(-2, origin_y(largest, 12), 24), BLUE)

def floor_h(largest):
    return -1 / 64 * largest

largest = 28
BLACK, WHITE, YELLOW, PINK = vec3(0, 0, 0), vec3(1, 1, 1), vec3(1,1,0), vec3(1,0.5,0.8)
RED, BLUE, PURPLE = vec3(0.9, 0, 0.1), vec3(0, 0.5, 1), vec3(0.2, 0.2, 0.6)

scene = Scene()
scene.set_directional_light((-1, 1, 1), 0.2, (1, 1, 1))
scene.set_background_color((1, 1, 1))
scene.set_floor(height=floor_h(largest), color=(1, 1, 1))  # set to the largest ball -1/64*r
initialize_voxels()
scene.finish()

[Taichi] Starting on arch=cuda

Camera:
* Drag with your left mouse button to rotate
* Press W/A/S/D/Q/E to move

[0. 0. 2.] [0. 0. 0.]


In [ ]:
x = ti.field(float, shape=(3, 3))
a = np.arange(9).reshape(3, 3).astype(np.int32)
x.from_numpy(a)
print(x)
type(x)

In [ ]:
arr = x.to_numpy()
print(arr)
type(arr)

In [ ]:
field = ti.Vector.field(3, int, shape=(256, 512))
field.shape  # (256, 512)
field.n      # 3

array = field.to_numpy()
array.shape  # (256, 512, 3)
print(type(array))

field.from_numpy(array)  # the input array must in the shape (256, 512, 3)
print(type(field))

## Render